# 删除测试

是否能按照预期删除索引存储的记录。

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 2.15 s, sys: 264 ms, total: 2.41 s
Wall time: 2.03 s


## 准备工作

In [2]:
%%time

from llama_index.core import Document

documents = [
    Document( text="test1", doc_id="doc_01"),
    Document( text="test2", doc_id="doc_02"),
    Document( text="test3", doc_id="doc_03"),
]

documents[0].id_

CPU times: user 0 ns, sys: 72 µs, total: 72 µs
Wall time: 73.9 µs


'doc_01'

In [3]:
%%time

!rm -rf storage

CPU times: user 808 µs, sys: 7.56 ms, total: 8.37 ms
Wall time: 113 ms


In [4]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()

CPU times: user 235 ms, sys: 27.9 ms, total: 263 ms
Wall time: 1.54 s


## 测试删除

In [5]:
%%time

status=index.delete_ref_doc("doc_03", delete_from_docstore=True)

CPU times: user 151 µs, sys: 0 ns, total: 151 µs
Wall time: 154 µs


In [7]:
index.ref_doc_info.items()

dict_items([('doc_01', RefDocInfo(node_ids=['06fc718a-d3be-4b26-8da6-6177d3febc74'], metadata={})), ('doc_02', RefDocInfo(node_ids=['789f27e7-644c-4529-8e77-3c0fe6fd17ba'], metadata={}))])

## 重新加载索引

In [10]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(
    persist_dir="./storage",
)
index = load_index_from_storage(storage_context)

In [11]:
index.ref_doc_info.items()

dict_items([('doc_01', RefDocInfo(node_ids=['06fc718a-d3be-4b26-8da6-6177d3febc74'], metadata={})), ('doc_02', RefDocInfo(node_ids=['789f27e7-644c-4529-8e77-3c0fe6fd17ba'], metadata={})), ('doc_03', RefDocInfo(node_ids=['8626c83e-c562-47cc-a977-83aa51f620eb'], metadata={}))])